## 本程序绘制计算的吸收谱、傅里叶变换和小波变换图， 服务对象是“_untar_fealab”文件夹

### 将本程序安放在“种子文件_single_xyz_WT”（名称可更改）所在文件夹（一级目录）下的任意一个文件夹下（二级目录）。
### 这里读取的文件夹为“_untar_fealab”中的文件
### 绘制出来的图都在“_untar_fealab_plot”中

last update: 2022.5.28

相对于27，将部分参数提前到前面

contacts: zhaohf@ihep.ac.cn

# 配置环境

## 导入各种模块

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import datetime

输出导入模板的版本。

In [2]:
print('python version:',sys.version)
print('numpy version:',np.__version__)
print('panda version:',pd.__version__)
print('matplotlib version:',mpl.__version__)
#print('math version:',math.__version__)
#print('datetime version:',datetime.__version__)

python version: 3.9.12 (main, Apr  4 2022, 05:22:27) [MSC v.1916 64 bit (AMD64)]
numpy version: 1.21.5
panda version: 1.4.2
matplotlib version: 3.5.1


# 参数设置

## 设置源/工作文件夹

In [3]:
# 设置源文件夹，即经过计算集群计算的文件夹
dir_src = 'AuPd-4000_au150-0_DW_R0_cncr_WT_untar_fealab'
# 构建提取文件夹和目录
dir_work = dir_src + '_plot'

## 设置需要绘制的feature

In [4]:
feature_plot = ['xmu','chi','ft','bft','wt']

## 设置统计原始还是已经经过pick的label

In [5]:
# pick_count = False : count all labels
#            = True  : count labels having been picked. (work with check)
pick_count = True
# 如果pick_out = True， 则需要设置如下需要pick out的label情况
dir_check = 'check'
file_checknum = 'check_num.txt'
# 设置输出文件名字前缀，如经过筛选的，建议pick，若没有（即pick_cout = False）则不读
prf_pick = 'pick'

## 其他一些参数

In [6]:
trivial = 0.0001

## 设置多少分之一的数据会作图

In [7]:
factor = 20
start = 1

## 设置绘图中legend

In [8]:
title = 'Au L3 EXAFS'

## 设置运行日志文件名

In [9]:
file_log = 'run.log'

# 函数、类

## 设置绘图格式和保存图片的模块

In [10]:
# To plot pretty figures
# %matplotlib具体作用是调用matplotlib.pyplot的绘图函数plot()进行绘图时，
# 或生成一个figure画布时，可以直接在你的python console里面生成图像。
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)   # matplotlib.rc 什么意思没找到
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

def save_fig(IMAGES_PATH, fig_id, tight_layout=True, fig_extension="png", resolution=300):
    os.makedirs(IMAGES_PATH, exist_ok=True)
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)
#    plt.show()

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

## 二维绘图

In [11]:
# 绘制x、y曲线，path和file分别是保存图片的路径和文件名
def plot_2d(x, y, path, file, title = 'Au-L3 EXAFS',
                              xlabel = 'E(eV)',
                              ylabel = '$\mu$(a.u.)',
                              location = 'upper right'):
#    import matplotlib.pyplot as plt

    #fig = plt.figure(num=1, figsize=(15, 8),dpi=80) #开启一个窗口，同时设置大小，分辨率
    fig = plt.figure()
    ax1 = fig.add_subplot(1,1,1)
    ax1.plot(x,y, label=file)
    
    ax1.set_title(title)
    ax1.set_xlabel(xlabel)
    ax1.set_ylabel(ylabel)
#ax1.set_xlim(-5,5) 
#ax1.set_ylim(-10,10)    
    ax1.legend(loc=location)
    save_fig(path,file)
#    os.makedirs(path, exist_ok=True)
#    path = os.path.join(path, file + ".png")
#    fig.savefig(path, format='png', dpi=300)
    plt.close(fig)

## 小波变换绘图——3D

In [12]:
# 绘制（k,r,wt）,path和file是保存的路径和图
def plot_3D(k,r,wt,path,file):
#    from mpl_toolkits import mplot3d
#    %matplotlib inline
#    import matplotlib.pyplot as plt
#    import numpy as np
#
    fig = plt.figure()
#    ax = plt.axes(projection='3d')
    ax=fig.add_subplot(1,1,1,projection='3d')
    ax.contour3D(k, r, wt, 50, cmap='binary')
#    ax.contour3D(k, r, wt, 50)
    ax.set_xlabel('$k(\AA^{-1})$')
    ax.set_ylabel('r($\AA$)')
    ax.set_zlabel('wt')
#调整观察角度和方位角。这里将俯仰角设为60度，把方位角调整为35度
#    ax.view_init(60, 35)
    ax.view_init(60, 205)
#  保存图片
    save_fig(path,file)
    plt.close(fig)

## 小波变换绘图——投影

In [13]:
#
def plot_prj(k,r,wt,path,file):
    from matplotlib import pyplot as plt
    from mpl_toolkits.mplot3d import Axes3D

#定义坐标轴
    fig4 = plt.figure()
#    ax4 = plt.axes(projection='3d')
    ax4 = fig4.add_subplot(1,1,1,projection='3d')

#生成三维数据

#作图
    ax4.plot_surface(k,r,wt,alpha=0.0,cmap='winter')     #生成表面， alpha 用于控制透明度
    ax4.contour(k,r,wt_mag,zdir='z', offset=-3,cmap="rainbow")  #生成z方向投影，投到x-y平面
#ax4.contour(k,r,wt_mag,zdir='x', offset=-6,cmap="rainbow")  #生成x方向投影，投到y-z平面
#ax4.contour(k,r,wt_mag,,zdir='y', offset=6,cmap="rainbow")   #生成y方向投影，投到x-z平面
#ax4.contourf(X,Y,Z,zdir='y', offset=6,cmap="rainbow")   #生成y方向投影填充，投到x-z平面，contourf()函数

#设定显示范围
    ax4.set_xlabel('$k(\AA^{-1})$')
#ax4.set_xlim(3, 20)  #拉开坐标轴范围显示投影
    ax4.set_ylabel('r($\AA$)')
#ax4.set_ylim(0,10)
    ax4.set_zlabel('wt')
#ax4.set_zlim(-3, 3)
#
#调整观察角度和方位角。这里将俯仰角设为60度，把方位角调整为35度
#ax.view_init(60, 35)
#ax.view_init(10, 10)
    save_fig(path,file)
    plt.close(fig4)

## 小波变换图——等高线

In [14]:
# 绘制（k,r,wt）,path和file是保存的路径和图
def plot_cntf(k,r,wt,path,file):
#    from mpl_toolkits import mplot3d
#    %matplotlib inline
#    import matplotlib.pyplot as plt
#    import numpy as np
#
    fig = plt.figure()
#    ax = plt.axes(projection='3d')
    ax=fig.add_subplot(1,1,1)
#    ax.contourf(k, r, wt, 50, cmap='binary')
    ax.contourf(k, r, wt, 50)
    ax.set_xlabel('$k(\AA^{-1})$')
    ax.set_ylabel('r($\AA$)')
#  保存图片
    save_fig(path,file)
    plt.close(fig)

## 将mu、chi、wt等绘制在一个图上

In [15]:
def plot_mu_chi_wt(e,mu,k,chi,k_wt,r_wt,wt,path,file):
#    from mpl_toolkits import mplot3d
#    %matplotlib inline
#    import matplotlib.pyplot as plt
#    import numpy as np
#
    fig = plt.figure()
#    fig, ax1,ax2,ax3,ax4 = plt.subplots(nrows=2, ncols=2)
    ax1 = fig.add_subplot(2,2,1)
    ax2 = fig.add_subplot(2,2,2)
    ax3 = fig.add_subplot(2,2,3,projection='3d')
    ax4 = fig.add_subplot(2,2,4)
#
# omega, mu
    ax1.plot(e, mu, label='$\mu$')
#    ax1.set_title(title)
    ax1.set_xlabel('E(eV)')
    ax1.set_ylabel('$\mu$(a.u.)')
#ax1.set_xlim(-5,5) 
#ax1.set_ylim(-10,10)    
    ax1.legend(loc='upper right')
#
#  k,chi*k^2
    ax2.plot(k, chi, label='$k^2\chi$')
#    ax1.set_title(title)
    ax2.set_xlabel('$k(\AA^{-1})$')
    ax2.set_ylabel('$k^2\chi$(a.u.)')
    ax2.legend(loc='upper right')
#
#   k_wt, r, wt_mag, 3D

#    ax3.contour3D(k, r, wt, 50, cmap='binary')
    ax3.contour3D(k_wt, r, wt, 50)
    ax3.set_xlabel('$k(\AA^{-1})$')
    ax3.set_ylabel('r($\AA$)')
    ax3.set_zlabel('wt')
#调整观察角度和方位角。这里将俯仰角设为60度，把方位角调整为35度
    ax3.view_init(60, 205)
#
#  k_wt,r,wt_mag
#    ax4.contourf(k, r, wt, 50, cmap='binary')
    ax4.contourf(k_wt, r, wt, 50)
    ax4.set_xlabel('$k(\AA^{-1})$')
    ax4.set_ylabel('r($\AA$)')
#
    plt.title('Au-L3 EXAFS')
    save_fig(path,file)
    plt.close(fig)

## 将mu、chi、k^2_chi、ft、bft、wt等绘制在一个图上

In [16]:
def plot_mu_chi_ft_bft_wt(e,mu,k,chi,k2chi,r_ft,ft,k_bft,bft,k_wt,r_wt,wt,path,file):
#    from mpl_toolkits import mplot3d
#    %matplotlib inline
#    import matplotlib.pyplot as plt
#    import numpy as np
#
    fig = plt.figure(figsize=(6,8))
#    fig, ax1,ax2,ax3,ax4 = plt.subplots(nrows=2, ncols=2)
    ax1 = fig.add_subplot(3,2,1)
    ax2 = fig.add_subplot(3,2,2)
    ax3 = fig.add_subplot(3,2,3)
    ax4 = fig.add_subplot(3,2,4)
    ax5 = fig.add_subplot(3,2,5)
#    ax5 = fig.add_subplot(3,2,5,projection='3d')
    ax6 = fig.add_subplot(3,2,6)
#
# omega, mu
    ax1.plot(e, mu, label='$\mu$')
#    ax1.set_title(title)
    ax1.set_xlabel('E(eV)')
    ax1.set_ylabel('$\mu$(a.u.)')
#ax1.set_xlim(-5,5) 
#ax1.set_ylim(-10,10)    
    ax1.legend(loc='upper right')
#
#  k,chi
    ax2.plot(k, chi, label='$\chi$')
#    ax1.set_title(title)
    ax2.set_xlabel('$k(\AA^{-1})$')
    ax2.set_ylabel('$\chi$(a.u.)')
    ax2.legend(loc='upper right')
#
#  k,chi*k^2
    ax3.plot(k, k2chi, label='$k^2\chi$')
#    ax1.set_title(title)
    ax3.set_xlabel('$k(\AA^{-1})$')
    ax3.set_ylabel('$k^2\chi$(a.u.)')
    ax3.legend(loc='upper right')
#
#  r_wt, ft_mag
    ax4.plot(r_ft, ft, label='FT')
    ax4.set_xlabel('r($\AA$)')
    ax4.set_ylabel('FT(a.u.)')
    ax4.legend(loc='upper right')
#
#  k_wt, bft
    ax5.plot(k_bft, bft, label='BFT')
    ax5.set_xlabel('$k(\AA^{-1})$')
    ax5.set_ylabel('$k^2\chi$(a.u.)')
    ax5.legend(loc='upper right')
#
#  k_wt,r_wt,wt_mag
#    ax5.contour3D(k_wt, r_wt, wt, 50, cmap='binary')
#    ax5.contour3D(k_wt, r_wt, wt, 50)
#    ax5.set_xlabel('$k(\AA^{-1})$')
#    ax5.set_ylabel('r($\AA$)')
#    ax5.set_zlabel('wt')
#调整观察角度和方位角。这里将俯仰角设为60度，把方位角调整为35度
#    ax5.view_init(60, 205)
#
#  k_wt,r_wt,wt_mag
#    ax6.contourf(k_wt, r_wt, wt, 50, cmap='binary')
    ax6.contourf(k_wt, r_wt, wt, 50)
    ax6.set_xlabel('$k(\AA^{-1})$')
    ax6.set_ylabel('r($\AA$)')
#
#    plt.title('Au-L3 EXAFS')
    plt.title('wavelet')
    save_fig(path,file)
    plt.close(fig)

## feature文件读取

### 读取wt/wtre/wtim.dat文件中的数组

In [17]:
def read_wt_np(path_expWT):
    """
    读取expWT.dat中k, R, I的数据，做成np数组形式，便于画图
    """
    exist = os.path.exists(path_expWT)
    if exist:
        expWT = pd.read_csv(path_expWT,delim_whitespace=True,header=None)
        expWT_data = expWT.values
        N_line = expWT_data.shape[0]
        a = expWT_data[0,0]
        for i in range(1,N_line):
            if expWT_data[i,0] == a :
                break
        nk = i
        nr = N_line // nk
        if N_line != nk * nr:
            print()
            print('The dimension of ',i, 'th dataset is bad!!')
            print()
#
#  define k, r list
#        print(type(expWT_data))
#        print(expWT_data.shape)
#        print(expWT_data)
#        print(expWT_data[0][0])
#        print(expWT_data[1][0])
#        print(expWT_data[2][0])
#        print(expWT_data[0:4, 0])
#        print(expWT_data[0,0:4])
#        print()
#        print()
#
        k_np = expWT_data[:nk,0]
#  本来就是 class 'numpy.ndarray' type
#        k_np = np.array(k)abs
#
        r = []
        for ir in range(nr):
            i = ir * nk
            r.append(expWT_data[i,1])
#
#  这个需要转换，原来r是list类型
        r_np = np.array(r)
#
#  define wt
#
#        print(k)
#        print(type(k))
#        print(r)
#        print(type(r))
#        print(r_np)
#        print(type(r_np))
        wt_np = expWT_data[:,2].reshape(nr,-1)
#        print(type(wt_re_np))
#        print(wt_re_np)
#        print(wt_re_np.shape)
#
        return k_np,r_np,wt_np
#
    else:
        return None  

### 读取expBFT.dat中的数据

这个程序也用于读取chi_FEFF.dat中的数据

In [18]:
def read_expBFT(path):
    """
    读取expBFT.dat中k、ft的数据
    """
    exist = os.path.exists(path)
    if exist:
        expBFT = pd.read_csv(path,delim_whitespace=True,header=None)
        expBFT_data = expBFT.values
        N = expBFT_data.shape[0]
        k = []
        bft = []
        for i in range(N):
            k.append(expBFT_data[i,0])
            bft.append(expBFT_data[i,1])
        return k, bft
    else:
        return None

### 对check文件夹下的数据进行重整，方便绘图

In [19]:
def check_rerange(list_check, list_plot = ['xmu','chi_FEFF','expWT','expFT','expBFT']):
    """
    对list_check中的文件，按照list_plot给出的关键词进行分组重排
    输出类似为[[coor.dat & chi_FEFF.dat .... for site 1][coor.dat & chi_FEFF.dat .... for site 2]...]
    以及[site1, site2, ...]的list
    """
#
# 找到表示位点的数字，存到site中。
    site_all = []
    for i,key in enumerate(list_plot):
        site = []
        len_key = len(key)
        for i, file in enumerate(list_check):
            result = os.path.splitext(file)
            file_pre = result[0]
            if file_pre[:len_key+1] == key + '_':
                site.append(file_pre[len_key+1:])
        site_all.append(site)
#
    tmp = site_all[0]
    for i in range(len(list_plot)-1):
        tmp = [a for a in site_all[i+1] if a in tmp]
    list_number = tmp
#
        
        
    list_check = []
    for i in range(len(list_number)):
        list_check.append([])
#
    for i, number in enumerate(list_number):
        for j, key in enumerate(list_plot):
            for k, file in enumerate(list_check):
                if key + '_' + number in file:
                    list_check[i].append(file)
                    break
#
    return list_number, list_check
#

# 程序正文

## 设置工作文件夹

In [20]:
import os
import sys
# 当前目录
path = os.path.abspath('./')
path

'D:\\haifeng\\work_20211031\\XAS-ML\\code\\datasets setup\\jupyterbook'

In [21]:
# 上一级目录
path_par = os.path.abspath(os.path.pardir)
path_par

'D:\\haifeng\\work_20211031\\XAS-ML\\code\\datasets setup'

In [22]:
# 设置源文件夹，即经过计算集群计算的文件夹
#dir_src = 'AuPd-4000_au150-0_DW_R0_cncr_WT_untar_treat'
path_src = os.path.join(path_par, dir_src)

In [23]:
# 检查源文件夹是否存在，存在则列出其内部文件夹，不存在则打印错误
exist = os.path.exists(path_src)
if exist:
    list_seed = os.listdir(path_src)
else:
    print()
    print('** Error!! WT untared directory', dir_src, 'is not found!' )
    print()
#    sys.exit()
list_seed

['100',
 '1000',
 '1100',
 '1200',
 '1300',
 '1400',
 '1500',
 '1600',
 '1700',
 '1800',
 '1900',
 '200',
 '2000',
 '2100',
 '2200',
 '2300',
 '2400',
 '2500',
 '2600',
 '2700',
 '2800',
 '2900',
 '300',
 '3000',
 '3100',
 '3200',
 '3300',
 '3400',
 '3500',
 '3600',
 '3700',
 '3800',
 '3900',
 '400',
 '4000',
 '500',
 '600',
 '700',
 '800',
 '900',
 'modify.log',
 'modify2.log',
 'pick.log']

In [24]:
# 去除list_seed中不是文件夹的部分
file_del = []
for dir_seed in list_seed:
    path_seed_src = os.path.join(path_src, dir_seed)
    if os.path.isfile(path_seed_src):
        file_del.append(dir_seed)
for file in file_del:
    list_seed.remove(file)

 ## 输出文件夹路径

In [25]:
# 构建提取文件夹和目录
#dir_work = dir_src + '_plot'
path_work = os.path.join(path_par, dir_work)
os.makedirs(path_work, exist_ok=True)
path_work

'D:\\haifeng\\work_20211031\\XAS-ML\\code\\datasets setup\\AuPd-4000_au150-0_DW_R0_cncr_WT_untar_fealab_plot'

## 创建日志文件

In [26]:
#file_log = 'modify.log'
path_file_log = os.path.join(path_work, file_log)
flog = open(path_file_log, 'w')

## 单张图形绘制

In [27]:
starttime = datetime.datetime.now()
print(f"==== Single figure ====")
print(f"==== Single figure ====", file=flog)
print()
print(file=flog)
print(f"== Start: {starttime}")
print(f"== Start: {starttime}", file=flog)
print()
print(file=flog)
#
for j, feature in enumerate(feature_plot):
    starttime1 = datetime.datetime.now()
    print(f" Start feature : {feature}, {j+1} / {len(feature_plot)} : {starttime1}")
    print(f" Start feature : {feature}, {j+1} / {len(feature_plot)} : {starttime1}", file=flog)
#
    for dir_seed in list_seed:
        print(f"dir_seed = {dir_seed}")
        print(f"dir_seed = {dir_seed}", file=flog)
#  在新文件夹内，新建与源seed一致的文件夹
        path_seed_work = os.path.join(path_work, dir_seed)
        os.makedirs(path_seed_work, exist_ok=True)
# 源seed的路径
        path_seed_src = os.path.join(path_src, dir_seed)
# 检查seed下是否存在目标文件夹
        dir_dest = feature
        path_dest_src = os.path.join(path_seed_src, dir_dest)
        exist = os.path.exists(path_dest_src)
        if exist:
# 源seed下目标文件夹下有多少目标文件  
            list_dest = os.listdir(path_dest_src)
#
            result = os.path.splitext(list_dest[0])
# 统计是否扣除check文件夹中的label
            if pick_count:
                path_dir_check = os.path.join(path_seed_src, dir_check)
                path_file_checknum = os.path.join(path_dir_check, file_checknum)
                exist_checknum = os.path.exists(path_file_checknum)
                if exist_checknum:
# csv读取方式
                    number_pd = pd.read_csv(path_file_checknum,header=None)
                    number_list = list(number_pd.values)
                    for i, number in enumerate(number_list):
                        number_list[i] = str(int(max(abs(number+trivial),abs(number-trivial))))
                    for i, number in enumerate(number_list):
                        file_label_num = dir_dest + '_' + number + result[1]
                        list_dest.remove(file_label_num)
                else:
                    print()
                    print(f"** Warning! File '{file_checknum}' in seed {dir_seed} for feature '{feature}' NOT found!")
                    print(f"** All the feature will be counted!")
#               end if exist_checknum
            else:
                pass
#           end if pick_count
#
#  数据太多，选择比例进行绘制
            len_list_dest = len(list_dest)
            for i in range(math.ceil(len_list_dest / factor)):
                file_dest = list_dest[i * factor + start - 1]
                path_file_dest_src = os.path.join(path_dest_src, file_dest)
#
                if feature == 'chi':
                    k,chi = read_expBFT(path_file_dest_src)
#  k, chi
                    result = os.path.splitext(file_dest)
                    file_pre = result[0]
                    plot_2d(k, chi, path_seed_work, file_pre
                                    , title = title
                                    , xlabel = '$k(\AA^{-1})$'
                                    , ylabel = '$\chi$(a.u.)'
                                    )
#  k, chi*k2
                    file_dest2 = 'k2' + file_pre
                    k2 = np.multiply(np.array(k),np.array(k))
                    chik2 = np.multiply(np.array(chi),np.array(k2))
                    chik2_list = chik2.tolist()
                    plot_2d(k, chik2_list, path_seed_work, file_dest2
                                    , title = title
                                    , xlabel = '$k(\AA^{-1})$'
                                    , ylabel = '$k^2\chi$(a.u.)'
                                    )
                elif feature == 'xmu' or feature == 'ft' or feature == 'bft':
                    x, y = read_expBFT(path_file_dest_src)
                    result = os.path.splitext(file_dest)
                    file_pre = result[0]
                    if feature == 'xmu':
                        plot_2d(x,y,path_seed_work,file_pre
                                ,title = title
                               )
                    elif feature == 'ft':
                        plot_2d(x,y,path_seed_work,file_pre
                                        , title = title
                                        , xlabel = 'r($\AA$)'
                                        , ylabel = 'FT(a.u.)'
                                        )
                    elif feature == 'bft':
                        plot_2d(x,y,path_seed_work,file_pre
                                        , title = title
                                        , xlabel = '$k(\AA^{-1})$'
                                        , ylabel = 'BFT(a.u.)'
                                        )
                    else:
                        pass
                elif feature == 'wt':
                    k,r,wt = read_wt_np(path_file_dest_src)
#
                    result = os.path.splitext(file_dest)
                    file_pre = result[0]
                    plot_cntf(k,r,wt,path_seed_work,file_pre)
                else:
                    pass
    #
        else:
            print()
            print(f"** Error!! No directory {dir_dest} found in seed {dir_seed}")
            print()
#            sys.exit()
    endtime1 = datetime.datetime.now()
    print(f" Finish feature : {feature}, {j+1} / {len(feature_plot)} : {endtime1}")
    print(f"== Running time : {endtime1-starttime1}")
    print(f" Finish feature : {feature}, {j+1} / {len(feature_plot)} : {endtime1}", file=flog)
    print(f"== Running time : {endtime1-starttime1}", file=flog)
#
endtime = datetime.datetime.now()
#
print()
print(file=flog)
print(f"== Start: {starttime}")
print(f"== Start: {starttime}", file=flog)
print(f"== End: {endtime}")
print(f"== End: {endtime}", file=flog)
#
print(f"== Running time : {endtime-starttime}")
print(f"==#unning time : {endtime-starttime}", file=flog)
flog.close()

==== Single figure ====

== Start: 2022-05-28 09:21:07.478268

 Start feature : xmu, 1 / 5 : 2022-05-28 09:21:07.479290
dir_seed = 100
Saving figure xmu_1
Saving figure xmu_118
Saving figure xmu_136
Saving figure xmu_2
Saving figure xmu_39
Saving figure xmu_57
Saving figure xmu_75
Saving figure xmu_95
dir_seed = 1000
Saving figure xmu_1
Saving figure xmu_118
Saving figure xmu_136
Saving figure xmu_19
Saving figure xmu_37
Saving figure xmu_55
Saving figure xmu_73
Saving figure xmu_91
dir_seed = 1100
Saving figure xmu_1
Saving figure xmu_117
Saving figure xmu_135
Saving figure xmu_18
Saving figure xmu_37
Saving figure xmu_55
Saving figure xmu_74
Saving figure xmu_92
dir_seed = 1200
Saving figure xmu_1
Saving figure xmu_118
Saving figure xmu_136
Saving figure xmu_2
Saving figure xmu_38
Saving figure xmu_56
Saving figure xmu_74
Saving figure xmu_92
dir_seed = 1300
Saving figure xmu_1
Saving figure xmu_118
Saving figure xmu_136
Saving figure xmu_19
Saving figure xmu_37
Saving figure xmu_55


Saving figure xmu_92
dir_seed = 900
Saving figure xmu_1
Saving figure xmu_117
Saving figure xmu_135
Saving figure xmu_18
Saving figure xmu_37
Saving figure xmu_55
Saving figure xmu_74
Saving figure xmu_92
 Finish feature : xmu, 1 / 5 : 2022-05-28 09:22:17.354995
== Running time : 0:01:09.875705
 Start feature : chi, 2 / 5 : 2022-05-28 09:22:17.354995
dir_seed = 100
Saving figure chi_1
Saving figure k2chi_1
Saving figure chi_118
Saving figure k2chi_118
Saving figure chi_136
Saving figure k2chi_136
Saving figure chi_2
Saving figure k2chi_2
Saving figure chi_39
Saving figure k2chi_39
Saving figure chi_57
Saving figure k2chi_57
Saving figure chi_75
Saving figure k2chi_75
Saving figure chi_95
Saving figure k2chi_95
dir_seed = 1000
Saving figure chi_1
Saving figure k2chi_1
Saving figure chi_118
Saving figure k2chi_118
Saving figure chi_136
Saving figure k2chi_136
Saving figure chi_19
Saving figure k2chi_19
Saving figure chi_37
Saving figure k2chi_37
Saving figure chi_55
Saving figure k2chi_5

Saving figure k2chi_137
Saving figure chi_2
Saving figure k2chi_2
Saving figure chi_38
Saving figure k2chi_38
Saving figure chi_56
Saving figure k2chi_56
Saving figure chi_74
Saving figure k2chi_74
Saving figure chi_92
Saving figure k2chi_92
dir_seed = 2900
Saving figure chi_1
Saving figure k2chi_1
Saving figure chi_117
Saving figure k2chi_117
Saving figure chi_135
Saving figure k2chi_135
Saving figure chi_19
Saving figure k2chi_19
Saving figure chi_37
Saving figure k2chi_37
Saving figure chi_55
Saving figure k2chi_55
Saving figure chi_73
Saving figure k2chi_73
Saving figure chi_91
Saving figure k2chi_91
dir_seed = 300

** Warning! File 'check_num.txt' in seed 300 for feature 'chi' NOT found!
** All the feature will be counted!
Saving figure chi_1
Saving figure k2chi_1
Saving figure chi_117
Saving figure k2chi_117
Saving figure chi_135
Saving figure k2chi_135
Saving figure chi_18
Saving figure k2chi_18
Saving figure chi_36
Saving figure k2chi_36
Saving figure chi_54
Saving figure k2chi

Saving figure ft_19
Saving figure ft_37
Saving figure ft_55
Saving figure ft_73
Saving figure ft_91
dir_seed = 1100
Saving figure ft_1
Saving figure ft_117
Saving figure ft_135
Saving figure ft_18
Saving figure ft_37
Saving figure ft_55
Saving figure ft_74
Saving figure ft_92
dir_seed = 1200
Saving figure ft_1
Saving figure ft_118
Saving figure ft_136
Saving figure ft_2
Saving figure ft_38
Saving figure ft_56
Saving figure ft_74
Saving figure ft_92
dir_seed = 1300
Saving figure ft_1
Saving figure ft_118
Saving figure ft_136
Saving figure ft_19
Saving figure ft_37
Saving figure ft_55
Saving figure ft_73
Saving figure ft_91
dir_seed = 1400
Saving figure ft_1
Saving figure ft_117
Saving figure ft_135
Saving figure ft_19
Saving figure ft_37
Saving figure ft_55
Saving figure ft_73
Saving figure ft_91
dir_seed = 1500

** Warning! File 'check_num.txt' in seed 1500 for feature 'ft' NOT found!
** All the feature will be counted!
Saving figure ft_1
Saving figure ft_117
Saving figure ft_135
Savin

dir_seed = 1100
Saving figure bft_1
Saving figure bft_117
Saving figure bft_135
Saving figure bft_18
Saving figure bft_37
Saving figure bft_55
Saving figure bft_74
Saving figure bft_92
dir_seed = 1200
Saving figure bft_1
Saving figure bft_118
Saving figure bft_136
Saving figure bft_2
Saving figure bft_38
Saving figure bft_56
Saving figure bft_74
Saving figure bft_92
dir_seed = 1300
Saving figure bft_1
Saving figure bft_118
Saving figure bft_136
Saving figure bft_19
Saving figure bft_37
Saving figure bft_55
Saving figure bft_73
Saving figure bft_91
dir_seed = 1400
Saving figure bft_1
Saving figure bft_117
Saving figure bft_135
Saving figure bft_19
Saving figure bft_37
Saving figure bft_55
Saving figure bft_73
Saving figure bft_91
dir_seed = 1500

** Warning! File 'check_num.txt' in seed 1500 for feature 'bft' NOT found!
** All the feature will be counted!
Saving figure bft_1
Saving figure bft_117
Saving figure bft_135
Saving figure bft_18
Saving figure bft_36
Saving figure bft_54
Saving

Saving figure wt_95
dir_seed = 1000
Saving figure wt_1
Saving figure wt_118
Saving figure wt_136
Saving figure wt_19
Saving figure wt_37
Saving figure wt_55
Saving figure wt_73
Saving figure wt_91
dir_seed = 1100
Saving figure wt_1
Saving figure wt_117
Saving figure wt_135
Saving figure wt_18
Saving figure wt_37
Saving figure wt_55
Saving figure wt_74
Saving figure wt_92
dir_seed = 1200
Saving figure wt_1
Saving figure wt_118
Saving figure wt_136
Saving figure wt_2
Saving figure wt_38
Saving figure wt_56
Saving figure wt_74
Saving figure wt_92
dir_seed = 1300
Saving figure wt_1
Saving figure wt_118
Saving figure wt_136
Saving figure wt_19
Saving figure wt_37
Saving figure wt_55
Saving figure wt_73
Saving figure wt_91
dir_seed = 1400
Saving figure wt_1
Saving figure wt_117
Saving figure wt_135
Saving figure wt_19
Saving figure wt_37
Saving figure wt_55
Saving figure wt_73
Saving figure wt_91
dir_seed = 1500

** Warning! File 'check_num.txt' in seed 1500 for feature 'wt' NOT found!
** Al

## 绘制check中的数据

In [28]:
dir_check = 'check'
path_check_work = os.path.join(path_work, dir_check)
os.makedirs(path_check_work, exist_ok=True)

In [29]:
flog = open(path_file_log, 'a')
print()
print(file=flog)
print(f"==== Single figure in check====")
print(f"==== Single figure in check====", file=flog)
print()
print(file=flog)
starttime = datetime.datetime.now()
print(f"== Start: {starttime}")
print(f"== Start: {starttime}", file=flog)
print()
print(file=flog)
#
for dir_seed in list_seed:
#
    print(f"dir_seed = {dir_seed}")
#
# 源seed的路径
    path_seed_src = os.path.join(path_src, dir_seed)
# 检查seed下是否存在check文件夹
    dir_check = 'check'
    path_check_src = os.path.join(path_seed_src, dir_check)
    exist = os.path.exists(path_check_src)
    if exist:
#  在新文件夹内，新建与源seed一致的文件夹
        path_seed_work = os.path.join(path_check_work, dir_seed)
        os.makedirs(path_seed_work, exist_ok=True)
        list_check = os.listdir(path_check_src)
        for file in list_check:
            path_file_src = os.path.join(path_check_src, file)
            result = os.path.splitext(file)
            file_pre = result[0]
            if 'xmu_' in file_pre:
                omega,mu = read_expBFT(path_file_src)
                plot_2d(omega,mu,path_seed_work,file_pre
                    ,title = title
                       )
                continue
#
            if 'chi_' in file:
                k,chi = read_expBFT(path_file_src)
                file_true = file_pre
                plot_2d(k, chi, path_seed_work, file_true
                            , title = title
                            , xlabel = '$k(\AA^{-1})$'
                            , ylabel = '$\chi$(a.u.)'
                            )
#  k, chi*k2
                file_dest2 = 'k2' + file_true
                k2 = np.multiply(np.array(k),np.array(k))
                chik2 = np.multiply(np.array(chi),np.array(k2))
                chik2_list = chik2.tolist()
                plot_2d(k, chik2_list, path_seed_work, file_dest2
                            , title = title
                            , xlabel = '$k(\AA^{-1})$'
                            , ylabel = '$k^2\chi$(a.u.)'
                            )
                continue
#
            if 'wt_' in file:
                k,r,wt = read_wt_np(path_file_src)
                plot_cntf(k,r,wt,path_seed_work,file_pre)
                continue
#
            if 'bft_' in file:
                k_wt, expBFT = read_expBFT(path_file_src)
                plot_2d(k_wt,expBFT,path_seed_work,file_pre
                            , title = title
                            , xlabel = '$k(\AA^{-1})$'
                            , ylabel = 'BFT(a.u.)'
                            )
                continue
#
            if 'ft_' in file:
                r, ft = read_expBFT(path_file_src)
                plot_2d(r,ft,path_seed_work,file_pre
                            , title = title
                            , xlabel = 'r($\AA$)'
                            , ylabel = 'FT(a.u.)'
                            )
                continue
#
    else:
        print()
        print(f"**  No directory {dir_check} found in seed {dir_seed}")
        print()
#
# 删除 seed 空文件夹
#
    try:
        os.removedirs(path_seed_work)
    except:
        pass
#
# 删除 check 空文件夹
#
try:
    os.removedirs(path_check_work)
except:
    pass
#
endtime = datetime.datetime.now()
#
print()
print(file=flog)
print(f"== Start: {starttime}")
print(f"== Start: {starttime}", file=flog)
print(f"== End: {endtime}")
print(f"== End: {endtime}", file=flog)
#
print(f"== Running time : {endtime-starttime}")
print(f"==#unning time : {endtime-starttime}", file=flog)
flog.close()


==== Single figure in check====

== Start: 2022-05-28 09:31:24.060311

dir_seed = 100
Saving figure bft_104
Saving figure bft_143
Saving figure bft_25
Saving figure bft_77
Saving figure bft_80
Saving figure chi_104
Saving figure k2chi_104
Saving figure chi_143
Saving figure k2chi_143
Saving figure chi_25
Saving figure k2chi_25
Saving figure chi_77
Saving figure k2chi_77
Saving figure chi_80
Saving figure k2chi_80
Saving figure ft_104
Saving figure ft_143
Saving figure ft_25
Saving figure ft_77
Saving figure ft_80
Saving figure wt_104
Saving figure wt_143
Saving figure wt_25
Saving figure wt_77
Saving figure wt_80
Saving figure xmu_104
Saving figure xmu_143
Saving figure xmu_25
Saving figure xmu_77
Saving figure xmu_80
dir_seed = 1000
Saving figure bft_113
Saving figure chi_113
Saving figure k2chi_113
Saving figure ft_113
Saving figure wt_113
Saving figure xmu_113
dir_seed = 1100
Saving figure bft_25
Saving figure bft_70
Saving figure chi_25
Saving figure k2chi_25
Saving figure chi_70


## 合起来一起画mu、chi、k^2chi、ft、bft和wt

In [30]:
flog = open(path_file_log, 'a')
print()
print(file=flog)
print(f"==== 6 figures in 1 photo====")
print(f"==== 6 figures in 1 photo====", file=flog)
print()
print(file=flog)
starttime = datetime.datetime.now()
print(f"== Start: {starttime}")
print(f"== Start: {starttime}", file=flog)
print()
print(file=flog)
#
for dir_seed in list_seed:
#
    print(f"dir_seed = {dir_seed}")
#  在新文件夹内，新建与源seed一致的文件夹
    path_seed_work = os.path.join(path_work, dir_seed + '_6')
    os.makedirs(path_seed_work, exist_ok=True)
#
# 源seed的路径
    path_seed_src = os.path.join(path_src, dir_seed)
# 检查seed下是否存在目标文件夹
    list_choose = os.listdir(path_seed_src)
    list_plot = ['xmu','chi','wt','ft','bft']
#    check = [False for a in list_plot if a not in list_choose]
# 确认seed文件夹中存在list_dir_plot所列所有文件夹
    if set(list_plot) <= set(list_choose) :
# 确认xmu、chi_FEFF等文件夹下文件数一样
        dir_xmu = list_plot[0]   # ‘xmu’
        path_xmu_src = os.path.join(path_seed_src, dir_xmu)
        list_xmu = os.listdir(path_xmu_src)
        len_xmu = len(list_xmu)
        for dest in list_plot[1:]:
            path_dest_src = os.path.join(path_seed_src, dest)
            a = len(os.listdir(path_dest_src))
            if a != len_xmu:
                print()
                print(f"** Error!! number in directory {dir_dest0} is different to that in {dir_xmu}")
                print()
                sys.exit()
#
# 下面增加的，目的就是为了计算len_xmu
#******************
        result = os.path.splitext(list_xmu[0])
# 统计是否扣除check文件夹中的label
        if pick_count:
            path_dir_check = os.path.join(path_seed_src, dir_check)
            path_file_checknum = os.path.join(path_dir_check, file_checknum)
            exist_checknum = os.path.exists(path_file_checknum)
            if exist_checknum:
# csv读取方式
                number_pd = pd.read_csv(path_file_checknum,header=None)
                number_list = list(number_pd.values)
                for i, number in enumerate(number_list):
                    number_list[i] = str(int(max(abs(number+trivial),abs(number-trivial))))
                for i, number in enumerate(number_list):
                    file_label_num = dir_xmu + '_' + number + result[1]
                    list_xmu.remove(file_label_num)
            else:
                print()
                print(f"** Warning! File '{file_checknum}' in seed {dir_seed} for feature '{dir_xmu}' NOT found!")
                print(f"** All the feature will be counted!")
#               end if exist_checknum
        else:
            pass
#           end if pick_count
#
        len_xmu = len(list_xmu)
#*******************
#
#
# 开始到各个文件夹下找相应文件画图：
        for i in range(math.ceil(len_xmu / factor)):
            ith = i * factor + start - 1
            list_number = []
            for j, dest in enumerate(list_plot):
                path_dest_src = os.path.join(path_seed_src, dest)
                list_dest = os.listdir(path_dest_src)
#
# 下面增加的，目的就是为了改变list_dest
#******************
                result = os.path.splitext(list_dest[0])
# 统计是否扣除check文件夹中的label
                if pick_count:
                    path_dir_check = os.path.join(path_seed_src, dir_check)
                    path_file_checknum = os.path.join(path_dir_check, file_checknum)
                    exist_checknum = os.path.exists(path_file_checknum)
                    if exist_checknum:
# csv读取方式
                        number_pd = pd.read_csv(path_file_checknum,header=None)
                        number_list = list(number_pd.values)
                        for k1, number in enumerate(number_list):
                            number_list[k1] = str(int(max(abs(number+trivial),abs(number-trivial))))
                        for k1, number in enumerate(number_list):
                            file_label_num = dest + '_' + number + result[1]
                            list_dest.remove(file_label_num)
                    else:
                        print()
                        print(f"** Warning! File '{file_checknum}' in seed {dir_seed} for feature '{dest}' NOT found!")
                        print(f"** All the feature will be counted!")
        #               end if exist_checknum
                else:
                    pass
        #           end if pick_count
        #
                len_dest = len(list_dest)
#*******************
#
                file_ith = list_dest[ith]
                result = os.path.splitext(file_ith)
                file_pre = result[0]
                list_number.append(file_pre[len(dest)+1:])
                path_file_ith_src = os.path.join(path_dest_src, file_ith)
#  提取绘图信息
                if j == 0 :     # 'xmu'
                    omega,mu = read_expBFT(path_file_ith_src)
                elif j == 1 :   # 'chi'
                    k,chi = read_expBFT(path_file_ith_src)
                    k2 = np.multiply(np.array(k),np.array(k))
                    chik2 = np.multiply(np.array(chi),np.array(k2))
                    chik2_list = chik2.tolist()
                elif j == 2 :   # 'wt'
                    k_wt,r_wt,wt = read_wt_np(path_file_ith_src)
                elif j == 3 :   # 'ft'
                    r_ft, ft = read_expBFT(path_file_ith_src)
                else:           # 'bft'
                    k_bft, expBFT = read_expBFT(path_file_ith_src)
# 查一下list_number中的数字（字符）是否都一致
            n0 = list_number[0]
            for j,a in enumerate(list_number[1:]):
                if a != n0 :
                    print()
                    print(f"** Error!! photo from different sites in one figure!")
                    print(f" {list_plot[0]}_{n0} VS {list_plot[j+1]}_{a}")
                    print()
                    sys.exit()
#  绘总图
            file = dir_seed + '_' + n0 + '_all'
#            print(f"k = {k}")
#            print(f"len_k = {len(k)}")
#            print(f"len_chi = {len(chi)}")
#            print(f"len_chik2_list = {len(chik2_list)}")
            plot_mu_chi_ft_bft_wt(omega,mu,k,chi,chik2_list,r_ft,ft,k_bft,expBFT,k_wt,r_wt,wt,path_seed_work,file) 
#
    else:
        print()
        print(f"** Error!! some directories in {list_plot} are not included in seed {dir_seed} directory")
        print()
#        sys.exit()
#
# 删除 空文件夹
#
    try:
        os.removedirs(path_seed_work)
    except:
        pass
#
endtime = datetime.datetime.now()
#
print()
print(file=flog)
print(f"== Start: {starttime}")
print(f"== Start: {starttime}", file=flog)
print(f"== End: {endtime}")
print(f"== End: {endtime}", file=flog)
#
print(f"== Running time : {endtime-starttime}")
print(f"==#unning time : {endtime-starttime}", file=flog)
flog.close()


==== 6 figures in 1 photo====

== Start: 2022-05-28 09:33:15.969042

dir_seed = 100
Saving figure 100_1_all
Saving figure 100_118_all
Saving figure 100_136_all
Saving figure 100_2_all
Saving figure 100_39_all
Saving figure 100_57_all
Saving figure 100_75_all
Saving figure 100_95_all
dir_seed = 1000
Saving figure 1000_1_all
Saving figure 1000_118_all
Saving figure 1000_136_all
Saving figure 1000_19_all
Saving figure 1000_37_all
Saving figure 1000_55_all
Saving figure 1000_73_all
Saving figure 1000_91_all
dir_seed = 1100
Saving figure 1100_1_all
Saving figure 1100_117_all
Saving figure 1100_135_all
Saving figure 1100_18_all
Saving figure 1100_37_all
Saving figure 1100_55_all
Saving figure 1100_74_all
Saving figure 1100_92_all
dir_seed = 1200
Saving figure 1200_1_all
Saving figure 1200_118_all
Saving figure 1200_136_all
Saving figure 1200_2_all
Saving figure 1200_38_all
Saving figure 1200_56_all
Saving figure 1200_74_all
Saving figure 1200_92_all
dir_seed = 1300
Saving figure 1300_1_all



** Warning! File 'check_num.txt' in seed 2200 for feature 'ft' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 2200 for feature 'bft' NOT found!
** All the feature will be counted!
Saving figure 2200_1_all

** Warning! File 'check_num.txt' in seed 2200 for feature 'xmu' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 2200 for feature 'chi' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 2200 for feature 'wt' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 2200 for feature 'ft' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 2200 for feature 'bft' NOT found!
** All the feature will be counted!
Saving figure 2200_117_all

** Warning! File 'check_num.txt' in seed 2200 for feature 'xmu' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 2200 for feat


** Warning! File 'check_num.txt' in seed 2400 for feature 'xmu' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 2400 for feature 'chi' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 2400 for feature 'wt' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 2400 for feature 'ft' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 2400 for feature 'bft' NOT found!
** All the feature will be counted!
Saving figure 2400_72_all

** Warning! File 'check_num.txt' in seed 2400 for feature 'xmu' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 2400 for feature 'chi' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 2400 for feature 'wt' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 2400 for feature 'ft' NOT found!
** All


** Warning! File 'check_num.txt' in seed 3200 for feature 'xmu' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 3200 for feature 'chi' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 3200 for feature 'wt' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 3200 for feature 'ft' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 3200 for feature 'bft' NOT found!
** All the feature will be counted!
Saving figure 3200_117_all

** Warning! File 'check_num.txt' in seed 3200 for feature 'xmu' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 3200 for feature 'chi' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 3200 for feature 'wt' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 3200 for feature 'ft' NOT found!
** Al

Saving figure 3800_36_all

** Warning! File 'check_num.txt' in seed 3800 for feature 'xmu' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 3800 for feature 'chi' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 3800 for feature 'wt' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 3800 for feature 'ft' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 3800 for feature 'bft' NOT found!
** All the feature will be counted!
Saving figure 3800_54_all

** Warning! File 'check_num.txt' in seed 3800 for feature 'xmu' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 3800 for feature 'chi' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 3800 for feature 'wt' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 3800 for feat


** Warning! File 'check_num.txt' in seed 600 for feature 'ft' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 600 for feature 'bft' NOT found!
** All the feature will be counted!
Saving figure 600_117_all

** Warning! File 'check_num.txt' in seed 600 for feature 'xmu' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 600 for feature 'chi' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 600 for feature 'wt' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 600 for feature 'ft' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 600 for feature 'bft' NOT found!
** All the feature will be counted!
Saving figure 600_135_all

** Warning! File 'check_num.txt' in seed 600 for feature 'xmu' NOT found!
** All the feature will be counted!

** Warning! File 'check_num.txt' in seed 600 for feature 'chi'

## 对check文件夹中的数据合起来一起画mu、chi、k^2chi、ft、bft和wt

In [31]:
dir_check = 'check'
path_check_work = os.path.join(path_work, dir_check)
os.makedirs(path_check_work, exist_ok=True)

In [32]:
flog = open(path_file_log, 'a')
print()
print(file=flog)
print(f"==== 6 figures in 1 photo for check====")
print(f"==== 6 figures in 1 photo for check====", file=flog)
print()
print(file=flog)
starttime = datetime.datetime.now()
print(f"== Start: {starttime}")
print(f"== Start: {starttime}", file=flog)
print()
print(file=flog)
#
for dir_seed in list_seed:
#
    print(f"dir_seed = {dir_seed}")
#
# 源seed的路径
    path_seed_src = os.path.join(path_src, dir_seed)
# 检查seed下是否存在check文件夹
    dir_check = 'check'
    path_check_src = os.path.join(path_seed_src, dir_check)
    exist = os.path.exists(path_check_src)
    if exist:
#  在新文件夹内，新建与源seed一致的文件夹
        path_seed_work = os.path.join(path_check_work, dir_seed)
        os.makedirs(path_seed_work, exist_ok=True)
#
        list_plot = ['xmu','chi','wt','ft','bft']
        list_check = os.listdir(path_check_src)
        list_number, list_recheck = check_rerange(list_check, list_plot)
        for i, number in enumerate(list_number):
            for j, file in enumerate(list_recheck[i]):
                path_file_src = os.path.join(path_check_src, file)
                result = os.path.splitext(file)
                file_pre = result[0]
#  提取绘图信息
                if j == 0 :     # 'xmu'
                    omega,mu = read_expBFT(path_file_src)
                elif j == 1 :   # 'chi'
                    k,chi = read_expBFT(path_file_src)
                    k2 = np.multiply(np.array(k),np.array(k))
                    chik2 = np.multiply(np.array(chi),np.array(k2))
                    chik2_list = chik2.tolist()
                elif j == 2 :   # 'wt'
                    k_wt,r_wt,wt = read_wt_np(path_file_src)
                elif j == 3 :   # 'ft'
                    r_ft, ft = read_expBFT(path_file_src)
                else:           # 'expBFT'
                    k_bft, expBFT = read_expBFT(path_file_src)
#  绘总图
            file = dir_seed + '_' + number + '_all' 
            plot_mu_chi_ft_bft_wt(omega,mu,k,chi,chik2_list,r_ft,ft,k_bft,expBFT,k_wt,r_wt,wt,path_seed_work,file) 
#
    else:
        print()
        print(f"**  No directory {dir_check} found in seed {dir_seed}")
        print()
#
# 删除 seed 空文件夹
#
    try:
        os.removedirs(path_seed_work)
    except:
        pass
#
# 删除 check 空文件夹
#
try:
    os.removedirs(path_check_work)
except:
    pass
#
endtime = datetime.datetime.now()
#
print()
print(file=flog)
print(f"== Start: {starttime}")
print(f"== Start: {starttime}", file=flog)
print(f"== End: {endtime}")
print(f"== End: {endtime}", file=flog)
#
print(f"== Running time : {endtime-starttime}")
print(f"==#unning time : {endtime-starttime}", file=flog)
flog.close()


==== 6 figures in 1 photo for check====

== Start: 2022-05-28 09:41:00.712777

dir_seed = 100
Saving figure 100_104_all
Saving figure 100_143_all
Saving figure 100_25_all
Saving figure 100_77_all
Saving figure 100_80_all
dir_seed = 1000
Saving figure 1000_113_all
dir_seed = 1100
Saving figure 1100_25_all
Saving figure 1100_70_all
dir_seed = 1200
Saving figure 1200_113_all
Saving figure 1200_150_all
dir_seed = 1300
Saving figure 1300_113_all
dir_seed = 1400
Saving figure 1400_139_all
dir_seed = 1500

**  No directory check found in seed 1500

dir_seed = 1600
Saving figure 1600_104_all
Saving figure 1600_107_all
dir_seed = 1700
Saving figure 1700_59_all
dir_seed = 1800
Saving figure 1800_104_all
dir_seed = 1900
Saving figure 1900_134_all
dir_seed = 200
Saving figure 200_101_all
Saving figure 200_29_all
dir_seed = 2000
Saving figure 2000_84_all
dir_seed = 2100
Saving figure 2100_29_all
dir_seed = 2200

**  No directory check found in seed 2200

dir_seed = 2300
Saving figure 2300_104_all


# 下面是单独copy某个文档，直接放到本程序文件夹下运行的情况

In [33]:
path_xmu = './feff84/xmu.dat'
omega,e,k,mu,mu0,chi = read_xmu(path_xmu)

NameError: name 'read_xmu' is not defined

In [ ]:
plot_2d(omega,mu,'./','$\mu$_feff84')

In [ ]:
plot_2d(e,mu0,'./','$\mu_0$_feff84')

In [ ]:
plot_2d(k,chi,'./','chi_feff84')

In [ ]:
k2 = np.multiply(np.array(k),np.array(k))
chik2 = np.multiply(np.array(chi),np.array(k2))
chik2_list = chik2.tolist()
plot_2d(k,chik2_list,'./','k2chi_feff84')

In [ ]:
chik = np.multiply(np.array(chi),np.array(k))
chik_list = chik.tolist()
plot_2d(k,chik_list,'./','kchi_feff84')

In [ ]:
path_expWT = './expWT.dat'
k_wt,r,wt_re,wt_im,wt_mag,wt_max_min = read_expWT_np(path_expWT)

In [ ]:
wt_max_min

In [ ]:
plot_3D(k_wt,r,wt_mag,'./','wt_3d')

In [ ]:
plot_cntf(k_wt,r,wt_mag,'./','wt_cntf_R0=-10')

In [ ]:
plot_cntf(k_wt,r,wt_re,'./','wt_re_cntf_R0=-10')

In [ ]:
plot_cntf(k_wt,r,wt_im,'./','wt_im_cntf_R0=-10')

In [ ]:
path_expFT = './evax_R0=8.25(default)/expFT.dat'
r, expFT_re, expFT_im, expFT_mag = read_expFT(path_expFT)

In [ ]:
plot_2d(r,expFT_mag,'./','expFT_mag')

In [ ]:
path_expBFT = './evax_R0=8.25(default)/expBFT.dat'
k, expBFT = read_expBFT(path_expBFT)

In [ ]:
plot_2d(k,expBFT,'./','expBFT')

In [ ]:
plot_all(omega,mu,k,chik2_list,k_wt,r,wt_mag,'./','feff84_all')

In [ ]:
plot_all(omega,mu,k,chik2_list,k_wt,r,wt_mag,'./','feff85L_all')